In [2]:
from marker.models import create_model_dict
from typing import Any

model_dict: dict[str,Any] | None = None

if model_dict is None:
    print("Loading Models")
    model_dict = create_model_dict()
    print(list(model_dict.keys()))

Loading Models
Loaded layout model datalab-to/surya_layout on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded table recognition model vikp/surya_tablerec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
['layout_model', 'texify_model', 'recognition_model', 'table_rec_model', 'detection_model', 'ocr_error_model']


In [3]:
# utilities 
import json
from typing import Any

def save_md_file(file_name:str, content:str):
     with open(f"./output/{file_name}.md", mode="w") as f:
        f.write(content)
 
def save_str_file(file_name:str, content:str):
     with open(f"./output/{file_name}", mode="w") as f:
        f.write(content)

def save_json_file(file_name:str, content:Any):
     
    with open(f"./output/{file_name}.json", "w") as f:
        f.write(json.dumps(content, indent=4)) 


def save_html_file(file_name:str, content:Any):
     
    with open(f"./output/{file_name}.html", "w") as f:
        f.write(json.dumps(content, indent=4)) 



In [4]:
# Document object creation 

import os
import tempfile
import warnings
 
# New changes
from marker.models import create_model_dict
from marker.builders.document import DocumentBuilder
from marker.builders.layout import LayoutBuilder
from marker.builders.ocr import OcrBuilder
from marker.providers.pdf import PdfProvider
from marker.schema.document import Document
from marker.converters.pdf import PdfConverter
from marker.output import save_output

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = ("1")
 
warnings.filterwarnings("ignore", category=UserWarning) 
 
# tuple[str, dict[str, Image.Image], dict, list]:
if model_dict is None:
    print("Loading Models")
    model_dict = create_model_dict()
 


with open("./pdf/2024 Sales Presentation C6501-PPOs-1.pdf", mode="rb") as f:
    content = f.read()
with tempfile.NamedTemporaryFile(suffix=".pdf") as temp_pdf:
    temp_pdf.write(content)
    temp_pdf.seek(0)
    filename = temp_pdf.name  
    
   
    pdf_converter = PdfConverter(
                config=None,
                artifact_dict=model_dict,
                processor_list=None,
                renderer=None
            )
    rendered = pdf_converter(filename)
    # print("images")
    # print(rendered.images)
    save_output(rendered, "native_output","test")

 


Recognizing Text: 100%|██████████| 27/27 [03:08<00:00,  6.96s/it]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 4/4 [00:08<00:00,  2.09s/it]


In [9]:
save_output(rendered, "chart","new")

In [ ]:
# Creating a document

pdf_provider = PdfProvider(filepath=filename, config=None) 
layout_builder = LayoutBuilder(model_dict["layout_model"])
ocr_builder = OcrBuilder(recognition_model=model_dict["recognition_model"], detection_model=model_dict["detection_model"])
document_builder = DocumentBuilder()

document:Document = document_builder(pdf_provider, layout_builder, ocr_builder)   

In [4]:
from marker.output import save_output
pdf_converter = PdfConverter(
            config=None,
            artifact_dict=model_dict,
            processor_list=None,
            renderer=None
        )
rendered = pdf_converter(filename)
# print("images")
# print(rendered.images)
text, ext, images = save_output(rendered)
save_output(rendered, "chart","new")

FileNotFoundError: /tmp/tmp77y0q43o.pdf

In [8]:
from marker.renderers.json import JSONRenderer

renderer = JSONRenderer()
md = renderer(document).html

save_md_file("test", md)

TypeError: 'NoneType' object is not iterable

In [ ]:
# from marker.renderers.html import HTMLRenderer

# renderer = HTMLRenderer({"paginate_output": True})
# full_html = renderer(document).html

# save_html_file("test", full_html)

from html_renderer import LLAMAHTMLRenderer

renderer = LLAMAHTMLRenderer({"paginate_output": True})
full_html = renderer(document).paginated_images
print(full_html)

# save_str_file("test.txt", full_html)

{'page-0': {'_page_0_Picture_1.png': <PIL.Image.Image image mode=RGB size=208x87 at 0x764F3C1374A0>}, 'page-1': {'_page_1_Picture_1.png': <PIL.Image.Image image mode=RGB size=401x94 at 0x764F3C713C80>}, 'page-2': {'_page_2_Picture_1.png': <PIL.Image.Image image mode=RGB size=208x87 at 0x764F3C7121B0>}, 'page-3': {'_page_3_Picture_1.png': <PIL.Image.Image image mode=RGB size=401x90 at 0x764F3F406810>}, 'page-4': {'_page_4_Picture_1.png': <PIL.Image.Image image mode=RGB size=345x90 at 0x764F3F405160>}, 'page-5': {'_page_5_Picture_1.png': <PIL.Image.Image image mode=RGB size=398x93 at 0x764F3C7128D0>}, 'page-6': {'_page_6_Picture_1.png': <PIL.Image.Image image mode=RGB size=376x89 at 0x764F3C713440>}, 'page-7': {'_page_7_Picture_1.png': <PIL.Image.Image image mode=RGB size=340x91 at 0x764F3C42CC50>}, 'page-8': {'_page_8_Picture_1.png': <PIL.Image.Image image mode=RGB size=403x94 at 0x764F3C42F260>}, 'page-9': {'_page_9_Picture_1.png': <PIL.Image.Image image mode=RGB size=385x90 at 0x764F3

In [4]:
# PDF to LLAMA conversion 

import os
import tempfile
import warnings
 
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict



os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = (
    "1"  # For some reason, transformers decided to use .isin for a simple op, which is not supported on MPS
)
 
warnings.filterwarnings("ignore", category=UserWarning)  # Filter torch pytree user warnings


 
# tuple[str, dict[str, Image.Image], dict, list]:
if model_dict is None:
    print("Loading Models")
    model_dict = create_model_dict()
    print(list(model_dict.keys()))


with open("./test1.pdf", mode="rb") as f:
    content = f.read()
with tempfile.NamedTemporaryFile(suffix=".pdf") as temp_pdf:
    temp_pdf.write(content)
    temp_pdf.seek(0)
    filename = temp_pdf.name
    config = {
       "paginate_output": True,       
    }
    processors = [
        "marker.processors.blockquote.BlockquoteProcessor",
        "marker.processors.code.CodeProcessor",
        "marker.processors.debug.DebugProcessor",
        "marker.processors.document_toc.DocumentTOCProcessor",
        "marker.processors.equation.EquationProcessor",
        "marker.processors.footnote.FootnoteProcessor",
        "marker.processors.ignoretext.IgnoreTextProcessor",
        "marker.processors.line_numbers.LineNumbersProcessor",
        "marker.processors.list.ListProcessor",
        "marker.processors.page_header.PageHeaderProcessor",
        "marker.processors.sectionheader.SectionHeaderProcessor",
        "marker.processors.table.TableProcessor",
        "marker.processors.text.TextProcessor",
    ]

    pdf_converter = PdfConverter(
            config=config,
            artifact_dict=model_dict,
            processor_list=processors,
            renderer= "llama_json.LLAMAJSONRenderer"
    )
    rendered = pdf_converter(filename)
    pages = rendered.pages

    save_json_file("special", pages)
    # save_json_file("hm3_pages", pages)
    # print("images")
    # print(rendered.images)
    # text, ext, images = text_from_rendered(rendered)
    


Recognizing layout: 100%|██████████| 2/2 [00:44<00:00, 22.22s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:02<00:00,  2.96s/it]
